# Experiment Dashboard

Interactive dashboard for exploring experiments. Works as both a Jupyter notebook and a Voila web app.


In [ ]:
import os

import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import matplotlib.pyplot as plt

from notebook_utils import get_api_client, demo_mode_banner, display_error

# Connect to API (demo fallback if unavailable)
api_url = os.environ.get("API_URL") or os.environ.get("AMPRENTA_API_URL")
client, demo_mode = get_api_client(api_url=api_url)

if demo_mode or client is None:
    demo_mode_banner()
    print("API unavailable — demo mode enabled.")
else:
    print(f"Connected to {getattr(client, 'api_url', api_url)}")


In [ ]:
# Load experiments (demo fallback if API unavailable)
from dataclasses import dataclass


@dataclass
class _DemoExperiment:
    id: str
    name: str
    description: str | None = None
    omics_type: str | None = None


try:
    if demo_mode or client is None:
        experiments = [
            _DemoExperiment(
                id="demo-exp-001",
                name="Demo Experiment",
                description="Sample experiment shown when API is unavailable.",
                omics_type="transcriptomics",
            )
        ]
    else:
        # Prefer a bounded list call if supported
        try:
            experiments = client.experiments.list(limit=200)
        except TypeError:
            experiments = client.experiments.list()

    print(f"Loaded {len(experiments)} experiments")
except Exception as e:
    demo_mode = True
    demo_mode_banner()
    experiments = [
        _DemoExperiment(
            id="demo-exp-001",
            name="Demo Experiment",
            description="Sample experiment shown when API is unavailable.",
            omics_type="transcriptomics",
        )
    ]
    display_error("Failed to load experiments from API", details=repr(e))

# Create dropdown options
if experiments:
    experiment_options = []
    for exp in experiments:
        exp_id = getattr(exp, "id", None)
        exp_name = getattr(exp, "name", None) or "(unnamed)"
        label = f"{exp_name} ({str(exp_id)[:8]}...)" if exp_id else exp_name
        experiment_options.append((label, exp_id))
else:
    experiment_options = [("No experiments found", None)]


In [ ]:
# Create widgets
experiment_dropdown = widgets.Dropdown(
    options=experiment_options if experiments else [("No experiments", None)],
    description="Experiment:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="500px"),
)

output_area = widgets.Output()


def _find_experiment_in_list(experiment_id):
    for exp in experiments or []:
        if str(getattr(exp, "id", "")) == str(experiment_id):
            return exp
    return None


def _list_datasets_best_effort():
    if demo_mode or client is None:
        return []
    try:
        return client.datasets.list(limit=200)
    except TypeError:
        return client.datasets.list()


def update_dashboard(experiment_id):
    """Update dashboard when experiment is selected."""
    with output_area:
        clear_output(wait=True)

        if not experiment_id:
            print("Please select an experiment.")
            return

        try:
            # Fetch experiment details (or demo object)
            if demo_mode or client is None:
                exp = _find_experiment_in_list(experiment_id)
            else:
                exp = client.experiments.get(experiment_id)

            if exp is None:
                print("Experiment not found.")
                return

            exp_id = getattr(exp, "id", None)
            exp_name = getattr(exp, "name", None) or "(unnamed)"

            # Display experiment details
            print(f"## {exp_name}")
            if exp_id:
                print(f"\n**ID:** {exp_id}")
            if getattr(exp, "description", None):
                print(f"**Description:** {exp.description}")
            if getattr(exp, "omics_type", None):
                print(f"**Omics Type:** {exp.omics_type}")
            if getattr(exp, "created_at", None):
                print(f"**Created:** {exp.created_at}")

            # Dataset stats for this experiment
            print("\n### Dataset stats")
            if demo_mode or client is None:
                df_ds = pd.DataFrame(
                    [
                        {"dataset": "Demo Dataset A", "omics_type": "transcriptomics", "feature_count": 1200},
                        {"dataset": "Demo Dataset B", "omics_type": "proteomics", "feature_count": 350},
                    ]
                )
            else:
                datasets = _list_datasets_best_effort()
                exp_datasets = []
                for d in datasets or []:
                    exp_ids = getattr(d, "experiment_ids", None) or []
                    if exp_id and exp_id in exp_ids:
                        exp_datasets.append(d)

                rows = []
                for d in exp_datasets:
                    feature_ids = getattr(d, "feature_ids", None) or {}
                    try:
                        feature_count = sum(len(v or []) for v in feature_ids.values())
                    except Exception:
                        feature_count = None

                    rows.append(
                        {
                            "dataset": getattr(d, "name", None) or str(getattr(d, "id", ""))[:8],
                            "omics_type": str(getattr(d, "omics_type", "")),
                            "feature_count": feature_count,
                        }
                    )

                df_ds = pd.DataFrame(rows)

            if df_ds.empty:
                print("No datasets linked to this experiment (or none returned by API).")
                return

            display(df_ds)

            # Plot feature counts if available
            if df_ds["feature_count"].notna().any():
                fig, ax = plt.subplots(figsize=(9, 4))
                df_plot = df_ds.copy()
                df_plot["feature_count"] = df_plot["feature_count"].fillna(0)
                ax.bar(df_plot["dataset"].astype(str), df_plot["feature_count"].astype(float))
                ax.set_ylabel("# features")
                ax.set_title("Datasets linked to experiment")
                ax.tick_params(axis="x", rotation=45)
                ax.grid(True, axis="y", alpha=0.3)
                plt.tight_layout()
                plt.show()
            else:
                # Fallback: show counts by omics type
                fig, ax = plt.subplots(figsize=(7, 4))
                counts = df_ds.groupby("omics_type").size().sort_values(ascending=False)
                ax.bar(counts.index.astype(str), counts.values)
                ax.set_ylabel("# datasets")
                ax.set_title("Datasets by omics type")
                ax.tick_params(axis="x", rotation=30)
                ax.grid(True, axis="y", alpha=0.3)
                plt.tight_layout()
                plt.show()

        except Exception as e:
            display_error("Error loading experiment dashboard", details=repr(e))


# Link dropdown to update function
experiment_dropdown.observe(lambda change: update_dashboard(change["new"]), names="value")

# Display widgets
display(experiment_dropdown)
display(output_area)

# Initial update if experiment is selected
if experiment_dropdown.value:
    update_dashboard(experiment_dropdown.value)


## Usage

1. Select an experiment from the dropdown above
2. The dashboard will automatically update with experiment details and visualizations
3. To run as a Voila app: `voila experiment_dashboard.ipynb`
